1. 절대경로, 상대경로
2. read file in python
3. preprocessing in data science

# 데이터 읽기, 전처리

1) 절대경로
- 파일의 root부터 시작하여 목적지까지의 전체적인 경로
- 최상위 / 을 포함하고 있음

2) 상대경로
- 현재 디렉터리를 기준으로 상위, 하위 디렉터리를 이동하며 표현되는 경로
- 최상위 / 를 거치지 않고도 가능

    - . : 현재 디렉터리
    - .. : 상위 디렉터리


3. 디렉터리 변경/확인 및 파일 속성 관련 함수
- os 모듈 함수로 디렉터리 변경 가능
    1. [현재 사용하는 디렉터리] : os.getcwd()
    2. [디렉터리 변경] : os.chidr(path)
    3. [디렉터리 생성] : os.mkdir(path)

- 데이터 입출력  
    
    1.csv 파일 : pandas.read_csv("파일경로(이름).csv")
    2. excel 파일 : pd.read_excel("파일경로.xlsx", engine = 'openyxl'), pd. read_excel('./지하철 승하차 인원 정보.xlsx', sheet_name = '20년 2월')

- DataFrame으로 불러오기
    1. index_col (하나의 칼럼을 인덱스로) : pd.read_csv('data/SeoulFloating.csv',index_col='date').head()
    2. usecols (원하는 칼럼만 선별) :  pd.read_csv('data/SeoulFloating.csv',usecols=['date','hour']).head()

# 데이터 전처리

데이터 전처리 과정
1. 데이터 확인
2. 결측값 처리
3. 이상값 처리
4. 변수 가공

1. 데이터 확인

    *데이터 변수 확인
    1. 변수 정의 확인 : 어떤 정보를 가진 변수인지
    2. 변수 유형 확인 :
        1. 질적,범주형(순서 유무 따라 명목형/순서형)
        2. 양적,수치형(트렌드유뮤에 따라 연속형/이산형)
    3. 변수 데이터 형식 확인 : 날짜, 수치, 텍스트, 이미지 등 구분
    
    *원시 데이터의 평균값, 최빈값, 중간값 등 변수들의 분포를 산포도, 박스 플롯, 히스토그램 등의 그래프로 분포 확인 가능

2. 결측값 처리

* 결측값 = 데이터 수집/입력 과정에서 누락되는 값 (시각화 문제, 변수 간 왜곡 가능성 있음)
* 누락 데이터 확인
    1. info() : 데이터 프레임 요약 정보, 각 열의 유효값 보여줌
    2. value_counts(dropna=False) : 각 열의 결측값을 포함한 전체 데이터 확인 가능
    3. isnull() : 누락 데이터 True, 유효 데이터 False (주로 df.isnull().sum() 형식)
    4. notnull() : 유효 데이터 True, 누락 데이터 False
    *결측값 replace : df.replace('?', np.nan, inplace = "True")

    

* 삭제 : 결측값이 무작위로 발생한 경우에 사용
    1. 전체 삭제 : 결측값이 발생한 모든 관측치를 삭제
    2. 부분 삭제 : 모델에 포함시킬 변수들 중 결측값이 발생한 관측치를 삭제
ex) df.dropna(axis=1, thresh=500)

* 대체 : 평균값, 최빈값 등으로 주로 활용, fillna() 이용
    1. 일괄 대체 : 변수들의 평균값을 구해 일괄적으로 대체
        a = df["열1"].mean(axis=0)
        df["열1"].fillna(a,inplace=True) #원본객체를 변경할 시 inplace = True
    2. 유사 유형 대체 : 범주형 변수 활용해 유사 유형의 평균값으로 대체
        category_means = df.groupby('Category')['Value'].apply(lambda x: x.fillna(x.mean()))
        
    3. meathod 활용한 이웃 값 치환
        df["열2"].fillna(method="ffill",inplace=True)
        df["열3"].fillna(method='bfill",inplace=True)

3. 이상값 처리 (이상치 = 기존 데이터와 거리가 먼 데이터)

- 이상치 확인
    1. df.describe()
    2. BoxPlot 시각화
        for i in df.describe().columns:
            df[[i]].boxplot()
            plt.show()
    3. Z-score : 평균에서 얼마의 표준 편차만큼 벗어났는지
    4. Tukey Fences : 4분기 (25%) 기준으로 이상치 판단 <br>
        def find_outlier_by_Tukey_DF(data) :
            q1, q3 = np.percentile(data,[25,75])
            iqr = q3 - q1
            
            lower_ bound = q1 - (iqr*1.5)
            upper_bound = q3 + (iqr*1.5)
            
            mask = data[(data['feature'] > upper_bound) | (data['feature'] < lower_bound)].index
            return mask
        
        x_df = pd.DataFrame(x, columns=['feature'])
        find_outlier_by_Turkey_DF(x_df)
        
        x[find_itlier_by_Turkey(x)]

- 이상치 처리

    1. 전체 삭제 : human error, 단순 오타, 비현실적 응답, 처리 중 오류 시 사용
    2. 다른 값 대체 : 절대적 관측치 숫자 적어 삭제 시 신뢰성 문제, 대체나 예측 모델로 대처
    3. 변수화 : 이상값이 자연 발생 시, 파악 후 변수 추가해 모델 포함 가능
    4. 리샘플링 : 이상값이 자연 발생 시, 분리 해 모델 만들 수 있음

4. 변수 가공 : 도메인 지식과 기존 변수를 사용해 데이터에 정보를 추가하는 일련의 과정

    1. 구간화 (binning)
        연속 데이터를 일정 구간으로 나눠 (이산적인 값으로 나타냄) 차이 드러냄
    2. 더미 변수
        범주형 변수를 연속형 변수로 변환하기 위해 사용
        0 또는 1로 표현되는 더미 변수를 사용
    3. 변환
        기존 데이터 특성 또는 다른 정보 이용해 다른 데이터로 변환 및 분석
    4. 스케일링
        숫자의 상대적 크기 차이로 인한 왜곡 방지 위해, 정규화(데이터 값을 동일한 크기 기준으로 나눈 비율 표시) 필요
        - 변수 단위 변경이나 변수 분포가 편향돼있을 경우, 변수 간 관계가 잘 드러나지 않는 경우 사용
        - Log 함수나 제곱근(square root) 사용
        - 라이브러리 임포트
            from sklearn.preprocessing import strandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler  

- StandardScaler
: 각 feature의 평균을 0, 분산을 1로 변경해, 모든 특성들이 같은 스케일 가짐
    -> 정규분포를 따를 때 적합
    
    standard_scaler = StandardScaler()
    scaled_data['열1'] = standard_scaler.fit_transform(scaled_data[['열1']])

- MinMaxScaler
: 모든 feature가 0과1 사이에 위치
    -> 데이터가 서로 다른 비율의 속성으로 구성되었을 때
    -> 연산 속도 높이고 알고리즘 최적화하는데 효과적
    
    scaler = MinMaxScaler(feature_range=(0,1))

- RobustScaler : 모든 특성들이 median과 quartile을 사용해 같은 크기를 가짐
    -> 이상치에 영향을 받지 않음